# TODO: change batch_size to rels per image!!!
# TODO: dummy data -- all zeros? 
# TODO: bucket images by # rels (to reduce padding)?


In [ ]:
tf.transpose()

In [1]:
import numpy as np
from utils import loadmat, mat_to_tf
from model_tf import Model

import tensorflow as tf
from utils import load_sg_batcher
import numpy as np

epochs     = 20
batch_size = 10

obj_mat   = 'data/vrd/objectListN.mat'
rel_mat   = 'data/vrd/predicate.mat'
train_mat = 'data/vrd/annotation_train.mat'
test_mat  = 'data/vrd/annotation_test.mat'

w2v_file  = 'data/vrd/w2v.npy'
obj_npy   = 'data/vrd/obj_probs.npy'
rel_npy   = 'data/vrd/rel_feats.npy'

In [2]:
# Load data
obj_dict = {r:i for i,r in enumerate(loadmat(obj_mat)['objectListN'])}
rel_dict = {r:i for i,r in enumerate(loadmat(rel_mat)['predicate'])}

word2idx = {'obj':obj_dict, 'rel':rel_dict}
w2v = np.load(w2v_file)      # TODO: make sure this is in (dict -> matrix) form

obj_probs = np.load(obj_npy).item()
rel_feats = np.load(rel_npy).item()

train_mat = loadmat(train_mat)['annotation_train']
test_mat  = loadmat(test_mat)['annotation_test']
# D = mat_to_triplets(mat, word2idx)
train_data = mat_to_tf(train_mat, word2idx, obj_probs, rel_feats, batch_size)
t = 30
test_data  = [i[:t] for i in train_data]
train_data = [i[t:] for i in train_data]
#TODO!
# test_data  = mat_to_tf(test_mat,  word2idx, obj_probs, rel_feats, batch_size)

R_full = (np.concatenate(train_data[0]),
          np.concatenate(train_data[1]),
          np.concatenate(train_data[2]))

In [3]:
learning_rate = 0.1

# Run model
model = Model(w2v, n=100, k=70, lamb1=0.05, lamb2=0.001, init_noise=0.05)
ground_truth = model.get_ground_truth(batch_size)
#accuracy = model.get_accuracy(ground_truth)

cost = model.loss(ground_truth)
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
#train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

batch_to_feed = lambda I, J, K, op, rf: { \
    ground_truth['I']: I,
    ground_truth['J']: J,
    ground_truth['K']: K,
    ground_truth['obj_probs']: op,
    ground_truth['rel_feats']: rf,
    ground_truth['R_full']: R_full
}

ValueError: Dimensions must be equal, but are 70 and 10

In [ ]:
#TODO: CPU ONLY
# with session_init() as sess:
with tf.Session() as sess:
    merged = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter(summaries_dir + '/train', sess.graph)
    test_writer = tf.train.SummaryWriter(summaries_dir + '/test')
    tf.initialize_all_variables().run()

    for e in range(epochs):
        print 'Beginning epoch {}'.format(e)

        for db, data_batch in enumerate(train_data):
            if db % 100 == 0:
                # if db % 100000 == 0: # TODO
                #     summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
                #     test_writer.add_summary(summary, i)
                #     accs = []
                #     for test_batch in test_data:
                #         feed_test = batch_to_feed(*test_batch)
                #         accs.append(sess.run(accuracy, feed_dict=feed_test))
                #     acc = np.mean(accs)
                #     print ' => epoch {} batch {}  |  Acurracy: {}'.format(e, db, acc)
                #     if acc > best_acc:
                #         model.save_npy(sess, file_path=save_weights + '_{}-{}.npy'.format(e,db))

                feed_train = batch_to_feed(*data_batch)

                # Record train set summaries, and train
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_op], feed_dict=feed_train,
                                      options=run_options, run_metadata=run_metadata)
                # TODO look up docs ...
                train_writer.add_run_metadata(run_metadata, 'epoch {}, batch {}'.format(e, batch))
                print('Adding run metadata for', i)
            else:
                # Record Train
                summary, _ = sess.run([merged, train_step], feed_dict=feed_test)

            train_writer.add_summary(summary, i)


    train_writer.close()
    test_writer.close()

<br>
<br>
<br>
<br>
<br>

# Interactive stuff!

http://stackoverflow.com/questions/33633370/how-to-print-the-value-of-a-tensor-object-in-tensorflow

In [5]:
asdf = tf.reshape(np.array([1,2,3,4]), [2,2])
tf.maximum()

In [ ]:
tf.tile()   tf.Tensor

In [4]:
%qtconsole

In [34]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [9]:
# Enter an interactive TensorFlow Session.
x = tf.Variable([[1.0, 2.0], [3.0, 4.0]])
a = tf.constant([[3.0, 3.0], [2.0, 2.0]])

# Initialize 'x' using the run() method of its initializer op.
x.initializer.run()

# Add an op to subtract 'a' from 'x'.  Run it and print the result
sub = x[0,0] / a
print(sub.eval())
# ==> [-2. -1.]

[[ 0.33333334  0.33333334]
 [ 0.5         0.5       ]]


In [71]:
# Enter an interactive TensorFlow Session.
# x = tf.Variable([[[1.0, 2.0], [3.0, 4.0]]])
# y = tf.reshape(tf.Variable(np.eye(2)), [1,2,2])
# z = tf.Variable(y)
y = tf.constant(np.eye(2))

# Initialize 'x' using the run() method of its initializer op.
# x = y + z


# tf.Print([y], [y])

y *= 2
print sess.run(y)
# print y.get_shape()

# tf.Te
# x.initializer.run()
# y.initializer.run()
# tf.initialize_all_variables()

# y.initializer.run()
# print(x.eval())
# print sess.run(x)


[[ 2.  0.]
 [ 0.  2.]]


In [33]:
# Close the Session when we're done.
sess.close()